In [1]:
# collect all ids

from pathlib import Path

directory_path = Path("failed_logs")

ids = set()
for file_path in directory_path.iterdir():
    if file_path.is_file():
        ids.add(file_path.stem)

len(ids)

59

In [2]:
index_error = set()
for id in ids:
    with open("failed_logs/" + id + '.err') as file:
        for line in file:
            if 'IndexError' in line:
                index_error.add(id)
print(len(index_error))
print(index_error)

19
{'PDBBdock_26716210_6261', 'PDBBdock_26716210_10212', 'PDBBdock_26716210_5203', 'PDBBdock_26716210_6230', 'PDBBdock_26716210_5132', 'PoBudock_26716208_54', 'PoBudock_26716208_319', 'PDBBdock_26716210_7288', 'PDBBdock_26716210_1246', 'PDBBdock_26716210_9298', 'PDBBdock_26716210_8654', 'PDBBdock_26716210_4478', 'PDBBdock_26716210_4552', 'PDBBdock_26716210_2703', 'PDBBdock_26716210_5135', 'Astdock_26716209_59', 'PDBBdock_26716210_4595', 'PoBudock_26716208_196', 'PDBBdock_26716210_7390'}


In [3]:
oserror = set()
for id in ids:
    with open("failed_logs/" + id + '.err') as file:
        for line in file:
            if 'OSError' in line:
                oserror.add(id)
print(len(oserror))
print(oserror)

3
{'PoBudock_26716208_217', 'PoBudock_26716208_386', 'PDBBdock_26716210_10714'}


In [4]:
rosetta_errors = set()
unrec_res = set()
for id in ids:
    with open("failed_logs/" + id + '.err') as file:
        for line in file:
            if 'Unrecognized residue' in line:
                unrec_res.add(id)
rosetta_errors.update(unrec_res)
print(len(rosetta_errors), len(unrec_res))

16 16


In [5]:
variant = set()
for id in ids:
    with open("failed_logs/" + id + '.err') as file:
        for line in file:
            if 'with variant' in line:
                variant.add(id)
rosetta_errors.update(variant)
print(len(rosetta_errors), len(variant))
print(variant)

17 1
{'PoBudock_26716208_272'}


In [6]:
disulfide = set()
for id in ids:
    with open("failed_logs/" + id + '.err') as file:
        for line in file:
            if 'disulfide' in line:
                disulfide.add(id)
rosetta_errors.update(disulfide)
print(len(rosetta_errors), len(disulfide))

19 2


In [7]:
missing_atom = set()
for id in ids:
    with open("failed_logs/" + id + '.err') as file:
        for line in file:
            if 'missing_atom' in line:
                missing_atom.add(id)
rosetta_errors.update(missing_atom)
print(len(rosetta_errors), len(missing_atom))

24 5


In [8]:
oom = set()
for id in ids:
    with open("failed_logs/" + id + '.err') as file:
        for line in file:
            if 'Out Of Memory' in line or 'Bus error' in line:
                oom.add(id)
print(len(oom))
print(oom)

8
{'PDBBdock_26716210_8596', 'PDBBdock_26716210_2120', 'PoBudock_26716208_144', 'PoBudock_26716208_85', 'PoBudock_26716208_413', 'PoBudock_26716208_201', 'PoBudock_26716208_106', 'PDBBdock_26716210_8599'}


In [9]:
keyerror = set()
for id in ids:
    with open("failed_logs/" + id + '.err') as file:
        for line in file:
            if 'KeyError' in line:
                keyerror.add(id)
print(len(keyerror))
print(keyerror)

5
{'PoBudock_26716208_30', 'PoBudock_26716208_252', 'PoBudock_26716208_287', 'PoBudock_26716208_11', 'PoBudock_26716208_412'}


In [11]:
currently_collected = set()
currently_collected.update(keyerror, oom, rosetta_errors, oserror, index_error)
print(len(currently_collected), len(ids))
ids.difference(currently_collected)

59 59


set()

In [12]:
print('keyerror', len(keyerror))
print('oom', len(oom))
print('rosetta_errors', len(rosetta_errors))
print('oserror', len(oserror))
print('index_error', len(index_error))

keyerror 5
oom 8
rosetta_errors 24
oserror 3
index_error 19
